##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from bs4 import BeautifulSoup

import numpy as np
import os
import time
import urllib.request

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


### Load transcripts

Love Island scripts available here for two seasons, parse text using BeautifulSoup

In [3]:
f = open("transcripts.txt", "wb")

transcript_links = ["https://subslikescript.com/series/Love_Island-4770018", "https://subslikescript.com/series/Love_Island-8819906"] #, "https://subslikescript.com/series/Temptation_Island-9174576"]

for transcript_link in transcript_links:
  html = urllib.request.urlopen(transcript_link).read()
  soup = BeautifulSoup(html, 'html.parser')
  links = soup.find_all('a')

  for link in links:
      url = link['href']
      if "Island" in url:
          full_url = 'https://subslikescript.com' + url
          script_html = urllib.request.urlopen(full_url).read()
          soup = BeautifulSoup(script_html, 'html.parser')
          script = soup.find("div", {"class": "full-script"}).get_text(separator='\n').strip()
          f.write(script.encode('utf-8'))

f.close()

### Read the data

First, look in the text:

In [134]:
# change path_to_file to transcripts.txt, comment out to use Shakespeare file
path_to_file = "transcripts.txt"
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 172738 characters


In [135]:
# Take a look at the first 250 characters in text
print(text[:250])

Brace you're, America.
Get ready for love, lust, and
some very tiny swimsuits.
♪ Baby.
My heart is full of love ♪
welcome to "love island" and
an unforgettable summer love
here in Fiji.
That's our host Arielle, I think
she's running late.
We'll get b


In [136]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

91 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [137]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `preprocessing.StringLookup` layer:

In [138]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts form tokens to character IDs:

In [139]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[58, 59, 60, 61, 62, 63, 64], [81, 82, 83]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [140]:
chars_from_ids = preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [141]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [142]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [143]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [144]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(172738,), dtype=int64, numpy=array([31, 75, 58, ..., 14, 14, 14])>

In [145]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [146]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

B
r
a
c
e
 
y
o
u
'


In [147]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [148]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'B' b'r' b'a' b'c' b'e' b' ' b'y' b'o' b'u' b"'" b'r' b'e' b',' b' '
 b'A' b'm' b'e' b'r' b'i' b'c' b'a' b'.' b'\n' b'G' b'e' b't' b' ' b'r'
 b'e' b'a' b'd' b'y' b' ' b'f' b'o' b'r' b' ' b'l' b'o' b'v' b'e' b','
 b' ' b'l' b'u' b's' b't' b',' b' ' b'a' b'n' b'd' b'\n' b's' b'o' b'm'
 b'e' b' ' b'v' b'e' b'r' b'y' b' ' b't' b'i' b'n' b'y' b' ' b's' b'w'
 b'i' b'm' b's' b'u' b'i' b't' b's' b'.' b'\n' b'\xe2\x99\xaa' b' ' b'B'
 b'a' b'b' b'y' b'.' b'\n' b'M' b'y' b' ' b'h' b'e' b'a' b'r' b't' b' '
 b'i' b's' b' ' b'f' b'u'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [149]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"Brace you're, America.\nGet ready for love, lust, and\nsome very tiny swimsuits.\n\xe2\x99\xaa Baby.\nMy heart is fu"
b'll of love \xe2\x99\xaa\nwelcome to "love island" and\nan unforgettable summer love\nhere in Fiji.\nThat\'s our host '
b"Arielle, I think\nshe's running late.\nWe'll get back the her later.\nThis is your captain\nspeaking.\nPle"
b'ase prepare the cabin for\nlanding on "love island."\nOver the next four weeks,\nsexy singles will get i'
b"nto this\ntropical haven.\n\n\n\n\n\n\n\r\n     (adsbygoogle = window.adsbygoogle || []).push({});\r\n\nThey've pa"


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [150]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [151]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [152]:
dataset = sequences.map(split_input_target)

In [153]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"Brace you're, America.\nGet ready for love, lust, and\nsome very tiny swimsuits.\n\xe2\x99\xaa Baby.\nMy heart is f"
Target: b"race you're, America.\nGet ready for love, lust, and\nsome very tiny swimsuits.\n\xe2\x99\xaa Baby.\nMy heart is fu"


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [154]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [155]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [156]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [157]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [158]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 92) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [159]:
model.summary()

Model: "my_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  23552     
_________________________________________________________________
gru_4 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_4 (Dense)              multiple                  94300     
Total params: 4,056,156
Trainable params: 4,056,156
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [160]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [161]:
sampled_indices

array([47, 30, 60, 10, 26, 43, 61, 49, 63, 46, 83, 53, 39, 74, 38, 83,  1,
       24, 77, 16, 78, 46, 55, 87, 41, 65, 74, 35, 72, 81, 36, 87, 23, 28,
       30, 13, 86, 30,  9, 81, 88, 40, 17, 70, 57, 49, 47, 10, 89, 61, 10,
       50, 59,  7, 28, 29, 25, 49, 29,  4, 22,  6, 28, 52, 42, 84, 39, 30,
       56, 68, 65, 67, 17, 24, 87, 17, 19, 45, 56,  5,  9, 15, 56, 53, 77,
       87, 63,  9, 29, 42, 33, 39, 30, 17,  1, 46, 65, 52, 90,  7])

Decode these to see the text predicted by this untrained model:

In [162]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'because\nis that none of that matters because\nyou can choose any of the five girls\nyou can choose any'

Next Char Predictions:
 b'RAc(;NdTfQzXJqIz\n9t1uQZ\xc2\xa3LhqFoxG\xc2\xa38>A-}A\'x\xc3\x87K2m]TR(\xc3\x91d(Ub$>?:T?!7#>WM{JA[khj29\xc2\xa324P["\'0[Xt\xc2\xa3f\'?MDJA2\nQhW\xc3\xb3$'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [163]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [164]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 92)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.5211296


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [165]:
tf.exp(mean_loss).numpy()

91.9394

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [166]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [167]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [168]:
EPOCHS = 30

In [169]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
26/26 [==============================] - 3s 58ms/step - loss: 3.9490
Epoch 2/50
26/26 [==============================] - 2s 55ms/step - loss: 3.0355
Epoch 3/50
26/26 [==============================] - 2s 56ms/step - loss: 2.6106
Epoch 4/50
26/26 [==============================] - 2s 56ms/step - loss: 2.4039
Epoch 5/50
26/26 [==============================] - 2s 56ms/step - loss: 2.3093
Epoch 6/50
26/26 [==============================] - 2s 56ms/step - loss: 2.2254
Epoch 7/50
26/26 [==============================] - 2s 57ms/step - loss: 2.1437
Epoch 8/50
26/26 [==============================] - 2s 58ms/step - loss: 2.0549
Epoch 9/50
26/26 [==============================] - 2s 58ms/step - loss: 1.9682
Epoch 10/50
26/26 [==============================] - 2s 58ms/step - loss: 1.8840
Epoch 11/50
26/26 [==============================] - 2s 58ms/step - loss: 1.7979
Epoch 12/50
26/26 [==============================] - 2s 57ms/step - loss: 1.7199
Epoch 13/50
26/26 [==================

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](images/text_generation_sampling.png)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [170]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [171]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [172]:
start = time.time()
states = None
next_char = tf.constant(['ANNOUNCER:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ANNOUNCER: This is me as
well, sle phocking side wo see what
the tattoos. Thank you very
much. Paige, it is your first time
much. Paige, it is your first time
much. Paige, it is your first time
much. Paige, it is your first time
much. Paige, it is years one.
I will say what I need to say the same to you.
I
was in a jonner boxer. I have whit every
day, my mum said you maye you
are sleeping tonight!
Let's remavious.
This is just the beginning my
grasnession for Ollie.
Do you really standing much
at me bronde of horts seores. I have gone for
more looks which looks is important
an ond speaking from my heart and
thing is, I'm kind of VDAziey first bigh ark from
here.
I'm try and think wow, you
ready for love? I'm definitely
ready for love.
My time has come for you
to choose which boy you want
to couple up with...
Arielle: You can get to know
everyone. I had a ceasty
of a it fight boy Nas is wondering if anyone
first boy Nas is wondering if anyone
first boy Nas is wondering if anyone
first b

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [125]:
start = time.time()
states = None
next_char = tf.constant(['ANNOUNCER:', 'LOVE ISLAND', 'CHARLIE:', 'WELCOME', 'MONDAY'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ANNOUNCER:OR HOUSE, LIKE, MACKENZIE.\n>> I WOULD LOT THIS MORNING.\nI'M TRYING TO JUST, LIKE, NERVOUS.\nPHYSICALATELY.\nLOOK AT THIS POINT?\nHONESTLY, THIS IS A GAME.\nEXACTLY.\nHE HAS TO MAKE HER FEET SOME MORE.\nBECAUSE FOR ME.\nMORE TO THE WAY FALL?\nI'M FEELING THAT.\nBUT I'M OKAY WITH A KISSIFWANT.\nWE ARE SOMEONE HERE AND\nBEING REALLY COMETONT.\nSCE ARE YOU GUYS.\nJUST BECAUSE I FELT LIKE I WAS\nSEEING SOMEONE TO COME UP WITH IS\nNOT A TAITY.\nIT WAS JUST UNEXPECTED.\nI THOUGHT WHO HERE THAT\nYOU'RE GOING TO BE GOING\nOVER HERE WITH THE LADIES.\nI HOPE JALEN, I THINK I\nWANTED TO CHECK ONLY OVER\nYOU.\nOBVIOUSLY AND I HONESTLY IT'S HARD NOT\nTODETITION FOR ME.\nI HAVE NOT EVEN GO THAT UPSET MY\nFEEDS TO BE HARD.\nBECAUSE YOU HAVE THE\nHAD SPOT TO BE LIKE A HOTCEENING DAY\nHORDS IF THAT HOUSELF WITH THE\nONES THE POSITION THAT YOU HAVE\nNOT EVENTONER LIKE IMPORTANT TO\nMOIRA BUT CARRINGTON IS A DATE.\nI FEEL LIKE HE IS HUGTING HER, I\nDO LIKE, BUT I DON'T KNOW.\nYEA

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [126]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [127]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
CONVERSATION LIKE HER, YOU
CAUGHT MINE WHO JEALLS, YOU TALK ABOUT
HONEST GODAN BUT JOHNNY.
BUT I AL


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [128]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [129]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [130]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [131]:
model.fit(dataset, epochs=1)

153/153 [==============================] - 11s 54ms/step - loss: 2.9966


Or if you need more control, you can write your own complete custom training loop:

In [132]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.3257
Epoch 1 Batch 50 Loss 2.1815
Epoch 1 Batch 100 Loss 2.0967
Epoch 1 Batch 150 Loss 2.0187

Epoch 1 Loss: 2.1514
Time taken for 1 epoch 9.69 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.9943
Epoch 2 Batch 50 Loss 1.8719
Epoch 2 Batch 100 Loss 1.7497
Epoch 2 Batch 150 Loss 1.7278

Epoch 2 Loss: 1.8391
Time taken for 1 epoch 9.43 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6586
Epoch 3 Batch 50 Loss 1.6467
Epoch 3 Batch 100 Loss 1.5742
Epoch 3 Batch 150 Loss 1.6222

Epoch 3 Loss: 1.6257
Time taken for 1 epoch 9.49 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.5258
Epoch 4 Batch 50 Loss 1.5070
Epoch 4 Batch 100 Loss 1.4484
Epoch 4 Batch 150 Loss 1.4825

Epoch 4 Loss: 1.4829
Time taken for 1 epoch 9.23 sec
_________________________________________________________________________